In [1]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
from scipy.optimize import minimize
from scipy.stats.mstats import winsorize
from scipy.stats import expon
from scipy.stats import pareto
from scipy.stats import lognorm
from scipy.stats import norm
import matplotlib.pyplot as plt
import scipy.stats as stats

In [59]:
data = pd.read_stata('C:\\Users\\chans\\Desktop\\UpdatedEmpirics\\sample_GMM.dta')

In [99]:
psi_vec = {
    "Energy": {"DAC": 0.998, "China": 0.946},
    "Health": {"DAC": 0.988, "China": 0.984},
    "Education": {"DAC": 0.997, "China": 0.970},
    "Water Supply & Sanitation": {"DAC": 0.995, "China": 0.991},
    "Government & Civil Society": {"DAC": 0.984, "China": 0.937},
    "General Budget Support": {"DAC": 0.997, "China": 0.806},
    "Other Social Infrastructure & Services": {"DAC": 0.997, "China": 0.979},
    "Agriculture, Forestry, Fishing": {"DAC": 1, "China": 0.938},
    "Industry, Mining, Construction": {"DAC": 1, "China": 0.887},
    "Transport & Storage": {"DAC": 1, "China": 0.944},
    "Communications": {"DAC": 1, "China": 0.953},
    "General Environment Protection": {"DAC": 1, "China": 0.710},
    "General Economic, Commercial, Labor Affairs": {"DAC": 1, "China": 0.867}
}

R_vec = {
    "Agriculture, Forestry, Fishing": {"DAC": 1.009, "China": 1.025},
    "Industry, Mining, Construction": {"DAC": 1.011, "China": 1.039},
    "Transport & Storage": {"DAC": 1.010, "China": 1.033},
    "Energy": {"DAC": 1.013, "China": 1.040},
    "Communications": {"DAC": 1.009, "China": 1.031},
    "Health": {"DAC": 1.009, "China": 1.023},
    "Education": {"DAC": 1.009, "China": 1.026},
    "General Environment Protection": {"DAC": 1.013, "China": 1.030},
    "Water Supply & Sanitation": {"DAC": 1.011, "China": 1.027},
    "Government & Civil Society": {"DAC": 1.010, "China": 1.018},
    "General Budget Support": {"DAC": 1.011, "China": 1.045},
    "General Economic, Commercial, Labor Affairs": {"DAC": 1.015, "China": 1.038},
    "Other Social Infrastructure & Services": {"DAC": 1.012, "China": 1.020}
}

f_vec = {
    "Agriculture, Forestry, Fishing": {"dac grant": 1000, "dac loan": 2000, "china loan":3000},
    "Industry, Mining, Construction": {"dac grant": 1000, "dac loan": 2000, "china loan":3000},
    "Transport & Storage": {"dac grant": 10000, "dac loan": 200, "china loan":3000},
    "Energy": {"dac grant": 1000, "dac loan": 2000, "china loan":3000},
    "Communications": {"dac grant": 1000, "dac loan": 2000, "china loan":3000},
    "Health": {"dac grant": 1000, "dac loan": 2000, "china loan":3000},
    "Education": {"dac grant": 1, "dac loan": 2, "china loan":3},
    "General Environment Protection": {"dac grant": 1, "dac loan": 2, "china loan":3},
    "Water Supply & Sanitation": {"dac grant": 1, "dac loan": 2, "china loan":3},
    "Government & Civil Society": {"dac grant": 1, "dac loan": 2, "china loan":3},
    "General Budget Support": {"dac grant": 1, "dac loan": 2, "china loan":3},
    "General Economic, Commercial, Labor Affairs": {"dac grant": 1, "dac loan": 2, "china loan":3},
    "Other Social Infrastructure & Services": {"dac grant": 1, "dac loan": 2, "china loan":3}
}

xi_vec = {"Turkey":1.05}


In [100]:
def expected_project_size(chi, f_g, f_d, f_c, iso_code, sector, df_group, sec_fin, sigma=2.2, delta=0.05):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    Rtilde_d=(R_d-(1-psi_d)*chi)/(psi_d)-(1-delta)
    Rtilde_c=(R_c-(1-psi_c)*chi)/(psi_c)-(1-delta)
    
    if df_group in ('dac grant', 'dac loan'):
        psi=psi_d
        Rtilde=Rtilde_d
    elif df_group == 'china loan':
        psi=psi_c
        Rtilde=Rtilde_c
        
    common = xi*(sigma-1)/(psi*(Rtilde)*(xi-sigma))
    
    F_ps = None
    
    if df_group == 'dac grant':
        if sec_fin in ('Gtilde', 'G'):
            F_ps = f_g
            
        elif sec_fin in ('Dtilde_Gtilde', 'D_G', 'Ctilde_Dtilde_Gtilde'):
            F_ps = (f_g**((sigma-xi)/sigma)-f_d**((sigma-xi)/sigma))/(f_g**(-(xi/sigma))-f_d**(-(xi/sigma)))
            
        elif sec_fin in ('Ctilde_Gtilde', 'Dtilde_Ctilde_Gtilde', 'Dtilde_C_Gtilde', 'D_C_G'):
            numerator = (f_g**((sigma-xi)/sigma))*(Rtilde_d**(((sigma-xi)*(sigma-1))/sigma))-(f_c**((sigma-xi)/sigma))*(Rtilde_c**(((sigma-xi)*(sigma-1))/sigma))
            denominator = (f_g**(-(xi/sigma)))*(Rtilde_d**((-xi*(sigma-1))/sigma))-(f_c**(-(xi/sigma)))*(Rtilde_c**((-xi*(sigma-1))/sigma))
            F_ps= (numeratr/denominator)/(Rtilde_d**(sigma-1))
            
    elif df_group == 'dac loan':
        if sec_fin in ('D_G', 'Dtilde_Gtilde'):
            F_ps = f_d
            
        elif sec_fin in ('Ctilde_Dtilde_Gtilde'):
            numerator = f_d**((sigma-xi)/sigma)-((f_c-f_d)**((sigma-xi)/sigma))*(((Rtilde_c**(sigma-1))/(Rtilde_d**(sigma-1)-Rtilde_c**(sigma-1)))**((sigma-xi)/sigma))
            denominator = f_d**((-xi)/sigma)-((f_c-f_d)**((-xi)/sigma))*(((Rtilde_c**(sigma-1))/(Rtilde_d**(sigma-1)-Rtilde_c**(sigma-1)))**((-xi)/sigma))
            F_ps = numerator/denominator
        
        elif sec_fin in ('D_C_G', 'Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde'):
            F_ps = (f_d-f_c)*((Rtilde_c**(sigma-1))/(Rtilde_c**(sigma-1)-Rtilde_d**(sigma-1)))
            
    elif df_group == 'china loan':
        if sec_fin in ('C_Gtilde', 'Ctilde_Gtilde', 'C_G'):
            F_ps = f_c
            
        elif sec_fin in ('D_C_G', 'Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde'):
            numerator = f_c**((sigma-xi)/sigma)-((f_d-f_c)**((sigma-xi)/sigma))*(((Rtilde_d**(sigma-1))/(Rtilde_c**(sigma-1)-Rtilde_d**(sigma-1)))**((sigma-xi)/sigma))
            denominator = f_c**((-xi)/sigma)-((f_d-f_c)**((-xi)/sigma))*(((Rtilde_d**(sigma-1))/(Rtilde_c**(sigma-1)-Rtilde_d**(sigma-1)))**((-xi)/sigma))
            F_ps = numerator/denominator
        
        elif sec_fin in ('Ctilde_Dtilde_Gtilde'):
            F_ps = (f_d-f_c)*((Rtilde_d**(sigma-1))/(Rtilde_d**(sigma-1)-Rtilde_c**(sigma-1)))
            
    exp_size = common * F_ps
    
    return exp_size

In [ ]:
def G_Ef(chi, f_g, f_d, f_c, iso_code, sector, sec_fin, sigma=2.2, delta=0.05):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    Rtilde_d=(R_d-(1-psi_d)*chi)/(psi_d)-(1-delta)
    Rtilde_c=(R_c-(1-psi_c)*chi)/(psi_c)-(1-delta)
    
    if sec_fin in ('D_G', 'Dtilde_Gtilde', 'G', 'Gtilde'):
        G_f=((f_g)**((sigma-xi)/(xi*(sigma-1))))/Rtilde_d
        
    elif sec_fin in ('Ctilde_Gtilde'):
        G_f=((f_g**((sigma-xi)/sigma))*(Rtilde_d**((xi*(1-sigma))/sigma))+(1-(Rtilde_c/Rtilde_d)**(sigma-1))*(f_c**((sigma-xi)/sigma))*(Rtilde_c**((xi*(1-sigma))/sigma)))**(sigma/(xi*(sigma-1)))
        
    elif sec_fin in ('DCG', 'Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde'):
        G_f=(((Rtilde_d**(1-sigma)-Rtilde_c**(1-sigma))**(xi/sigma))*((f_d-f_c)**((sigma-xi)/sigma))+(f_g**((sigma-xi)/sigma))*(Rtilde_d**((xi*(1-sigma))/sigma))+(1-(Rtilde_c/Rtilde_d)**(sigma-1))*(f_c**((sigma-xi)/sigma))*(Rtilde_c**((xi*(1-sigma))/sigma)))**(sigma/(xi*(sigma-1)))
        
    elif sec_fin in ('Ctilde_Dtilde_Gtilde'):
        G_f=(((Rtilde_c**(1-sigma)-Rtilde_d**(1-sigma))**(xi/sigma))*((f_c-f_d)**((sigma-xi)/sigma))+(f_g**((sigma-xi)/sigma))*(Rtilde_d**((xi*(1-sigma))/sigma)))**(sigma/(xi*(sigma-1)))
        
    return G_f

In [ ]:
def D_Y_s(chi, f_g, f_d, f_c, iso_code, sector, sec_fin, sigma=2.2, delta=0.05):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    Rtilde_d=(R_d-(1-psi_d)*chi)/(psi_d)-(1-delta)
    Rtilde_c=(R_c-(1-psi_c)*chi)/(psi_c)-(1-delta)
    
    if sec_fin in ('G', 'Gtilde'):
        D_df=0
        D_cf=0
        
    elif sec_fin in ('DG', 'Dtilde_Gtilde'):
        D_df=(f_d**((sigma-xi)/sigma))*((Rtilde_d)**((sigma-1)*(sigma-xi)/sigma))
        D_cf=0
        
    elif sec_fin in ('Ctilde_Gtilde'):
        D_df=0
        D_cf=(f_c**((sigma-xi)/sigma))*((Rtilde_c)**((sigma-1)*(sigma-xi)/sigma))
        
    elif sec_fin in ('D_C_G','Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde'):
        D_df=((f_d-f_c)**((sigma-xi)/sigma))*((Rtilde_c*Rtilde_d)**((sigma-1)*(sigma-xi)/sigma))*((1/(Rtilde_c**(sigma-1)-Rtilde_d**(sigma-1)))**((sigma-xi)/sigma))
        D_cf=(f_c**((sigma-xi)/sigma))*((Rtilde_c)**((sigma-1)*(sigma-xi)/sigma))-((f_d-f_c)**((sigma-xi)/sigma))*((Rtilde_c*Rtilde_d)**((sigma-1)*(sigma-xi)/sigma))*((1/(Rtilde_c**(sigma-1)-Rtilde_d**(sigma-1)))**((sigma-xi)/sigma))
        
    elif sec_fin in ('Ctilde_Dtilde_Gtilde'):
        D_df=(f_d**((sigma-xi)/sigma))*((Rtilde_d)**((sigma-1)*(sigma-xi)/sigma))-((f_c-f_d)**((sigma-xi)/sigma))*((Rtilde_c*Rtilde_d)**((sigma-1)*(sigma-xi)/sigma))*((1/(Rtilde_d**(sigma-1)-Rtilde_c**(sigma-1)))**((sigma-xi)/sigma))
        D_cf=((f_c-f_d)**((sigma-xi)/sigma))*((Rtilde_c*Rtilde_d)**((sigma-1)*(sigma-xi)/sigma))*((1/(Rtilde_d**(sigma-1)-Rtilde_c**(sigma-1)))**((sigma-xi)/sigma))
        
    G_f=G_Ef(chi, f_g, f_d, f_c, iso_code, sector, sec_fin, sigma=2.2, delta=0.05)
    
    D_d=(G_f**(xi*(1-sigma)/sigma))*D_df
    D_c=(G_f**(xi*(1-sigma)/sigma))*D_cf
    
    return D_d, D_c

In [ ]:
def calculate_fixed_costs(chi, iso_code, sector, sec_fin, empirical_g, empirical_d, empirical_c, sigma=2.2, delta=0.05):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - delta)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - delta)
    
    common_d = xi * (sigma - 1) / (psi_d * Rtilde_d * (xi - sigma))
    common_g = xi * (sigma - 1) / (psi_d * Rtilde_d * (xi - sigma))
    common_c = xi * (sigma - 1) / (psi_c * Rtilde_c * (xi - sigma))
    
    f_g, f_d, f_c = None, None, None

    if sec_fin in ('G', 'Gtilde'):
        f_g = empirical_g / common_g
    
    elif sec_fin in ('DG', 'Dtilde_Gtilde'):
        f_d = empirical_d / common_d
        
        def solve_f_g(f_g):
            return empirical_g / common_g - ((f_g**((sigma - xi) / sigma) - f_d**((sigma - xi) / sigma)) / 
                                             (f_g**(-xi / sigma) - f_d**(-xi / sigma)))
        
        f_g = fsolve(solve_f_g, empirical_g / common_g)[0]

    elif sec_fin in ('Ctilde_Gtilde'):
        f_c = empirical_c / common_c
        
        def solve_f_g(f_g):
            numerator = (f_g**((sigma - xi) / sigma)) * (Rtilde_d**(((sigma - xi) * (sigma - 1)) / sigma)) - \
                        (f_c**((sigma - xi) / sigma)) * (Rtilde_c**(((sigma - xi) * (sigma - 1)) / sigma))
            denominator = (f_g**(-xi / sigma)) * (Rtilde_d**((-xi * (sigma - 1)) / sigma)) - \
                          (f_c**(-xi / sigma)) * (Rtilde_c**((-xi * (sigma - 1)) / sigma))
            return empirical_g / common_g - numerator / denominator
        
        f_g = fsolve(solve_f_g, empirical_g / common_g)[0]
    
    elif sec_fin in ('D_C_G', 'Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde'):
        diff_d_c = (empirical_d / common_d) / (Rtilde_c**(sigma - 1) / (Rtilde_c**(sigma - 1) - Rtilde_d**(sigma - 1)))
        
        def solve_f_c(f_c):
            return (empirical_c / common_c - 
                    (f_c**((sigma - xi) / sigma) - 
                    (diff_d_c**((sigma - xi) / sigma)) * 
                    ((Rtilde_d**(sigma - 1) / (Rtilde_c**(sigma - 1) - Rtilde_d**(sigma - 1)))**((sigma - xi) / sigma))) / 
                    (f_c**(-xi / sigma) - 
                    (diff_d_c**(-xi / sigma)) * 
                    ((Rtilde_d**(sigma - 1) / (Rtilde_c**(sigma - 1) - Rtilde_d**(sigma - 1)))**(-xi / sigma))))
        
        f_c = fsolve(solve_f_c, empirical_c / common_c)[0]
        f_d = diff_d_c + f_c
        
        def solve_f_g(f_g):
            numerator = (f_g**((sigma - xi) / sigma)) * (Rtilde_d**(((sigma - xi) * (sigma - 1)) / sigma)) - \
                        (f_c**((sigma - xi) / sigma)) * (Rtilde_c**(((sigma - xi) * (sigma - 1)) / sigma))
            denominator = (f_g**(-xi / sigma)) * (Rtilde_d**((-xi * (sigma - 1)) / sigma)) - \
                          (f_c**(-xi / sigma)) * (Rtilde_c**((-xi * (sigma - 1)) / sigma))
            return empirical_g / common_g - numerator / denominator
        
        f_g = fsolve(solve_f_g, empirical_g / common_g)[0]

    elif sec_fin in ('Ctilde_Dtilde_Gtilde'):
        diff_c_d = (empirical_c / common_c) / (Rtilde_d**(sigma - 1) / (Rtilde_d**(sigma - 1) - Rtilde_c**(sigma - 1)))
        
        def solve_f_d(f_d):
            return (empirical_d / common_d - 
                    (f_d**((sigma - xi) / sigma) - 
                    (diff_c_d**((sigma - xi) / sigma)) * 
                    ((Rtilde_c**(sigma - 1) / (Rtilde_d**(sigma - 1) - Rtilde_c**(sigma - 1)))**((sigma - xi) / sigma))) / 
                    (f_d**(-xi / sigma) - 
                    (diff_c_d**(-xi / sigma)) * 
                    ((Rtilde_c**(sigma - 1) / (Rtilde_d**(sigma - 1) - Rtilde_c**(sigma - 1)))**(-xi / sigma))))
        
        f_d = fsolve(solve_f_d, empirical_d / common_d)[0]
        f_c = diff_c_d + f_d
        
        def solve_f_g(f_g):
            return empirical_g / common_g - ((f_g**((sigma - xi) / sigma) - f_d**((sigma - xi) / sigma)) / 
                                             (f_g**(-xi / sigma) - f_d**(-xi / sigma)))
        
        f_g = fsolve(solve_f_g, empirical_g / common_g)[0]
    
    return f_g, f_d, f_c

In [ ]:
def calculate_chi_bounds(iso_code):
    R_d_min = float('inf')
    R_c_min = float('inf')
    upper_bound_min = float('inf')

    for sector in sectors:
        R_d = R_vec[sector]['DAC']
        R_c = R_vec[sector]['China']
        psi_d = psi_vec[sector]['DAC']
        psi_c = psi_vec[sector]['China']
        
        # Lower bound is the minimum of R_d and R_c across all sectors
        R_d_min = min(R_d_min, R_d)
        R_c_min = min(R_c_min, R_c)
        
        # Upper bound calculation for each sector
        upper_bound_d = (R_d - psi_d * (1 - delta)) / (1 - psi_d)
        upper_bound_c = (R_c - psi_c * (1 - delta)) / (1 - psi_c)
        upper_bound_min = min(upper_bound_min, upper_bound_d, upper_bound_c)

    lower_bound = min(R_d_min, R_c_min)
    upper_bound = upper_bound_min
    
    return lower_bound, upper_bound

In [101]:
def dictify(params):
    # Convert flat parameter list back to a dictionary structure similar to f_vec
    it = iter(params)
    return {sector: {df_group: next(it) for df_group in f_vec[sector]} for sector in f_vec}

In [102]:
def calculate_weighting_matrix(empirical_moments):
    # Square the empirical moments
    squared_moments = np.square(empirical_moments)
    # Avoid division by zero by adding a small constant or using np.clip
    safe_squared_moments = np.clip(squared_moments, a_min=1e-10, a_max=None)
    # Construct diagonal matrix with reciprocals of squared moments
    W = np.diag(1 / safe_squared_moments)
    return W

In [103]:
def gmm_objective(f_vec, data):
    model_moments = []
    empirical_moments = data['commitment_conusd'].values
    W = calculate_weighting_matrix(empirical_moments)

    for index, row in data.iterrows():
        sector = row['sector']
        df_group = row['df_group']
        f_gg, f_dd, f_cc = f_vec[sector]['dac grant'], f_vec[sector]['dac loan'], f_vec[sector]['china loan']
        chi, sigma, delta = 1, 2.2, 0.05
        model_moment= expected_project_size(chi, f_gg, f_dd, f_cc, row['recipient'], row['sector'],
                                             row['df_group'], row['sec_fin'], sigma, delta)
        model_moments.append(model_moment)
    
    model_moments = np.array(model_moments)
    if np.any(np.isnan(model_moments)):
        return np.inf  # Handle NaNs by returning a large number, which is undesirable in optimization

    diff = model_moments - empirical_moments
    return np.dot(np.dot(diff.T, W), diff)  # Weighted squared errors

In [ ]:
def set_initial_conditions(chi, sector, sec_fin):
    # Default initial values
    initial_values = {'dac grant': 1, 'dac loan': 1, 'china loan': 1}
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    Rtilde_d=(R_d-(1-psi_d)*chi)/(psi_d)-(1-delta)
    Rtilde_c=(R_c-(1-psi_c)*chi)/(psi_c)-(1-delta)
    
    if sec_fin == 'Dtilde_Ctilde':
        initial_values = {'dac grant': 1, 'dac loan': 100, 'china loan': 200}
        
    elif sec_fin in ('Gtilde', 'G'):
        initial_values = {'dac grant': 1, 'dac loan': 1, 'china loan': 1}

    elif sec_fin in ('Dtilde_Gtilde', 'D_G'):
        initial_values = {'dac grant': 1, 'dac loan': 1, 'china loan': 1}

    elif sec_fin in ('Ctilde_Gtilde', 'C_G'):
        initial_values = {'dac grant': 1, 'dac loan': 1, 'china loan': 1}
            
    # Example generic setup
    initial_values = {'dac grant': 10, 'dac loan': 20, 'china loan': 30}

    
    return initial_values

In [104]:
initial_params = [f_vec[sector][source] for sector in f_vec for source in f_vec[sector]]
bounds = [(1, 500000000) for _ in initial_params]

In [105]:
result = minimize(lambda params: gmm_objective(dictify(params), data), 
                  initial_params, bounds=bounds, method='L-BFGS-B')

In [106]:
result

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: (33.69831245914821-0.008017011724243088j)
        x: [ 1.000e+03  2.000e+03 ...  2.000e+00  3.000e+00]
      nit: 1
      jac: [-4.263e-06 -1.421e-06 ... -4.974e-06  1.421e-06]
     nfev: 80
     njev: 2
 hess_inv: <39x39 LbfgsInvHessProduct with dtype=float64>